<a href="https://colab.research.google.com/github/yenlung/AI-Demo/blob/master/%E3%80%90Demo08%E3%80%91%E7%94%A8diffusers%E5%A5%97%E4%BB%B6%E7%94%9F%E6%88%90%E5%9C%96%E5%83%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖼️ 用 diffusers 套件生成圖像 (學習篇)

### 1. 安裝必要套件

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

In [ ]:
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import torch
import gc
import matplotlib.pyplot as plt

萬一需要 HuggingFace Token 時請這樣做

In [ ]:
# from huggingface_hub import login
# from google.colab import userdata
#
# hf_token = userdata.get('HuggingFace')
# login(token=hf_token)

### 2. 讀入模型

In [ ]:
model_name = "digiplay/majicMIX_realistic_v6"

為了固定 random seed, 請在這裡選一個吉祥的數字。

In [ ]:
N = 31327

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    #revision="fp16",  # for SD 1.5 and 2.1
    use_safetensors=True
).to("cuda")

### 3. 基本生圖

In [ ]:
# 解析度
height = 512
width = 768

# 推理步數
num_inference_steps = 50

# 指引強度
guidance_scale = 7.5

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
prompt = "a Taiwanese college student using her laptops in a cafe."

In [ ]:
# 固定 random seed
seed = N
generator = torch.Generator("cuda").manual_seed(seed)

In [ ]:
image_basic = pipe(
    prompt=prompt,
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=generator
).images[0]

In [ ]:
plt.imshow(image_basic)
plt.axis("off")

### 4. Prompt 優化

#### 4-1 加入 Negative Prompt

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
negative_prompt = "easynegative, human, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot"

In [ ]:
seed = N
generator = torch.Generator("cuda").manual_seed(seed)

In [ ]:
image_ng_prompt = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=generator
).images[0]

In [ ]:
plt.imshow(image_ng_prompt)
plt.axis("off")

#### 4-2 優化原有的 prompt

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
enhanced_prompt = prompt + ", masterpiece, ultra high quality, intricate skin details, cinematic lighting"

In [ ]:
seed = N
generator = torch.Generator("cuda").manual_seed(seed)

In [ ]:
image_ng_prompt = pipe(
    prompt=enhanced_prompt,
    negative_prompt=negative_prompt,
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=generator
).images[0]

In [ ]:
plt.imshow(image_ng_prompt)
plt.axis("off")

### 5. 換新的排程器

In [ ]:
# 換成 UniPCMultistepScheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

#### 如果需要, 重新修改 prompt

In [ ]:
prompt = "a Taiwanese college student using her laptops in a cafe."

enhanced_prompt = prompt + ", masterpiece, ultra high quality, intricate skin details, cinematic lighting"
negative_prompt = "easynegative, human, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worstquality, low quality, normal quality, jpegartifacts, signature, watermark, username, blurry, bad feet, cropped, poorly drawn hands, poorly drawn face, mutation, deformed, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, extra fingers, fewer digits, extra limbs, extra arms,extra legs, malformed limbs, fused fingers, too many fingers, long neck, cross-eyed,mutated hands, polar lowres, bad body, bad proportions, gross proportions, text, error, missing fingers, missing arms, missing legs, extra digit, extra arms, extra leg, extra foot"

In [ ]:
# 清除記憶體
gc.collect()
torch.cuda.empty_cache()

In [ ]:
seed = N
generator = torch.Generator("cuda").manual_seed(seed)

In [ ]:
image_strong = pipe(
        prompt=enhanced_prompt,
        negative_prompt=negative_prompt,
        height=height,
        width=width,
        num_inference_steps=20,
        #num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator
        ).images[0]

In [ ]:
plt.imshow(image_strong)
plt.axis("off")